In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# 1. Descripción del problema a resolver

En este cuadernillo se quiere cuantificar la brecha sanitaria que existe para diversos problemas
de salud relevantes para el Instituto Nacional del Tórax. En específico, se quiere saber si
tales problemas son resueltos por el sistema de salud o no. Para esto, se contrastará la información de 3 fuentes de información:

1. Casos teóricos por problema de salud según incidencia (ET).
2. Casos reales de egresos hospitalarios a nivel país por problema de salud (EP).
3. Casos reales de egresos hospitalarios del INT por problema de salud (EI).

En primer lugar, se calculará la brecha de egresos hospitalarios teóricos y a nivel país. O sea:

$Brecha Pais= ET - EP$

Si $ET > EP$, entonces existe una brecha, y el sistema sanitario ha fallado en resolver el problema de salud. En caso contrario, si se ha resuelto el problema de salud.

En nuestro caso, se quieren identificar los problemas de salud donde $Brecha Pais > 0$, ya que el INT podría hacerse cargo de tales casos sin resolver en el nuevo hospital.

Una vez identificado los problemas de salud donde hay capacidad de crecimiento, se calculará el porcentaje de crecimiento que tendría el INT al asumir el 100% de la brecha del país. Para esto, se utilizará la siguiente fórmula:

$Casos Nuevos del INT = EI + Brecha Pais$

$PorcentajeDeCrecimientoINT = \frac{Casos Nuevos del INT - EI}{EI} * 100$

Esto nos permitirá identificar qué diagnósticos necesitarán un mayor porcentaje de crecimiento que otros.

## Ejemplo de cálculo

A modo de ejemplo, se mostrará un cálculo manual:

$ET = 1055; EP = 493; EI = 220$

Con estos datos, podemos calcular que:

$Brecha Pais = 1055 - 493 = 562$

Por lo tanto, faltan **562** casos que el sistema sanitario falla en atender. Ahora, asumiendo que el INT se hará cargo del 100% de estos casos (sólo para temas de simplicidad. Sin embargo, en ciertos casos es imposible, especialmente tomando en cuenta la cantidad de casos que recibe actualmente el recinto), es posible calcular el % de crecimiento para esta brecha.

$Casos Nuevos del INT = 220 + 562 = 782$

$PorcentajeDeCrecimientoINT = \frac{782 - 220}{220} * 100 = 255\% $

Por lo tanto, el INT debería crecer en un 255% de su capacidad actual para suplir el 100% de la necesidad sanitaria en el diagnóstico X.


# 1.1 Cálculo de Brecha País con Egresos

## Extracción de egresos teóricos (ET)


In [3]:
# Carga la poblacion teorica de los 45 diagnosticos mas relevantes para el INT
poblacion_teorica = pd.read_excel(
    "../data/processed/proyeccion_problemas_de_salud.xlsx", sheet_name="Proyeccion problemas INE"
)

# Solamente deja el codigo CIE de los problemas de salud
poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]

# Extrae los codigos de los diagnosticos relevantes y los deja como indice
DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico").sort_index()

# De los casos totales, se asume que una parte (82%) es paciente FONASA. Ademas, se hara una
# correccion para estimar que son pacientes hospitalizados
RATIO_FONASA = 0.82
RATIO_HOSPITALIZADOS = 1
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

In [4]:
# Deja los egresos teoricos a nivel pais
poblacion_teorica_pais = poblacion_teorica.query("Estrato == 'Pais'").copy()

# Selecciona los egresos teoricos hasta 2020
poblacion_teorica_pais = poblacion_teorica_pais[[i for i in range(2017, 2021)]]

# Corrige la poblacion teorica para pacientes FONASA y hospitalizados
poblacion_teorica_pais = round(poblacion_teorica_pais * RATIO_FONASA * RATIO_HOSPITALIZADOS)

## Extracción de egresos a nivel país (EP)


In [5]:
# Lee egresos hospitalarios a nivel país
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

# Deja solamente los 45 egresos relevantes para el INT y desde 2017
egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Extracción de egresos INT (EI)


In [6]:
# Obtiene los egresos del Torax desde los egresos totales
egresos_int = egresos_pais.query("ESTABLECIMIENTO_SALUD == 112103").copy()

In [7]:
# Transforma la tabla de egresos del pais a formato de analisis
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
).sort_index()


# Transforma la tabla de egresos int a formato de analisis

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

Una vez extraído los datos y haberlos transformados para realizar su análisis, se procederá a calcular las respectivas brechas.

En primer lugar, se obtendrá la brecha a nivel país por problema de salud. Recordemos que la fórmula de cálculo es:

$Brecha Pais= ET - EP$


In [8]:
brecha_pais = (poblacion_teorica_pais - tabla_dinamica_egresos_pais).astype("Int32")
brecha_pais.style.map(visualize.color_negative_red)

,2017,2018,2019,2020
Diagnostico,,,,
C33X,14,22,24,27
C340,879,918,933,998
C341,670,583,588,785
C342,992,993,1029,1053
C343,784,768,802,878
C381,140,144,136,149
C384,-34,-39,-47,-37
C450,-76,-71,-70,-47
C780,669,682,651,775


Los resultados indican que en gran parte de los diagnósticos faltó suplir la demanada del problema
de salud (ej: C340, C341, C342, C343, etc). Los resultados se pueden dividir en distintos tipos
de
problema de salud:

1. **Cáncer de Pulmón (C33X a C782)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando C33X, C384, C450). EL diagnóstico con una mayor necesidad de atención fue
   el C342, con una falta de 1026 egresos.

2. **Patologías Valvulares (I051 a I712)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando I420).


Con estos resultados, es posible identificar todos los diagnósticos donde es necesario cerrar la brecha de atención de salud. Se seleccionaran los diagnósticos donde en al menos 1 de los años entre 2017 y 2020 exista una $Brecha País > 0.$


In [9]:
# Selecciona solamente los diagnosticos donde exista una brecha > 0 en alguno de los anios
brecha_pais_a_resolver = brecha_pais[(brecha_pais > 0).any(axis=1)]
# Obtiene los codigos de los diagnosticos con una brecha
diagnosticos_con_necesidad_sanitaria = brecha_pais_a_resolver.index

Una vez identificado los diagnosticos con necesidad sanitaria, es posible calcular cuánto debería crecer el INT en cada uno de estos diagnósticos para suplir tal brecha.


In [10]:
# Calcula los nuevos casos que deberia atender, asumiendo que se hara cargo del 100% de la brecha
casos_nuevos_a_atender_int = brecha_pais + tabla_dinamica_egresos_int

# Calcula el porcentaje de crecimiento, tomando en cuenta los casos actuales del INT
porcentaje_crecimiento_int = (
    casos_nuevos_a_atender_int - tabla_dinamica_egresos_int
) / tabla_dinamica_egresos_int

# Deja solamente los diagnosticos donde hay posibilidad de crecimiento en alguno de los anios
porcentaje_crecimiento_int_diags_a_atender = porcentaje_crecimiento_int.loc[
    diagnosticos_con_necesidad_sanitaria
]

In [11]:
display(porcentaje_crecimiento_int_diags_a_atender.style.format("{:,.1%}"))

,2017,2018,2019,2020
Diagnostico,,,,
C33X,"1,400.0%","2,200.0%",600.0%,385.7%
C340,"4,185.7%","5,400.0%","3,887.5%","14,257.1%"
C341,372.2%,265.0%,267.3%,"1,032.9%"
C342,"5,221.1%","4,137.5%","5,145.0%","15,042.9%"
C343,664.4%,619.4%,668.3%,"2,310.5%"
C381,933.3%,685.7%,715.8%,"2,980.0%"
C780,"1,173.7%","1,082.5%","1,050.0%","5,166.7%"
C782,"3,086.2%","1,647.2%","2,230.8%","1,766.7%"
D143,"2,700.0%","2,135.0%","3,289.3%","16,466.7%"


In [12]:
from IPython.display import display, HTML

HTML(
    build_features.obtener_tabla_resumen_egresos(
        None,
        tabla_dinamica_egresos_pais,
        poblacion_teorica_pais,
        brecha_pais,
    )
    .to_html()
    .replace("\\n", "<br>")
)

ANO_EGRESO,2017,2018,2019,2020
DIAG1,,,,
C33X,Pais: 25 Teorica:39 Brecha:14,Pais: 18 Teorica:40 Brecha:22,Pais: 17 Teorica:41 Brecha:24,Pais: 14 Teorica:41 Brecha:27
C340,Pais: 163 Teorica:1042 Brecha:879,Pais: 143 Teorica:1061 Brecha:918,Pais: 148 Teorica:1081 Brecha:933,Pais: 103 Teorica:1101 Brecha:998
C341,Pais: 372 Teorica:1042 Brecha:670,Pais: 478 Teorica:1061 Brecha:583,Pais: 493 Teorica:1081 Brecha:588,Pais: 316 Teorica:1101 Brecha:785
C342,Pais: 50 Teorica:1042 Brecha:992,Pais: 68 Teorica:1061 Brecha:993,Pais: 52 Teorica:1081 Brecha:1029,Pais: 48 Teorica:1101 Brecha:1053
C343,Pais: 258 Teorica:1042 Brecha:784,Pais: 293 Teorica:1061 Brecha:768,Pais: 279 Teorica:1081 Brecha:802,Pais: 223 Teorica:1101 Brecha:878
C381,Pais: 41 Teorica:181 Brecha:140,Pais: 41 Teorica:185 Brecha:144,Pais: 52 Teorica:188 Brecha:136,Pais: 42 Teorica:191 Brecha:149
C384,Pais: 39 Teorica:5 Brecha:-34,Pais: 44 Teorica:5 Brecha:-39,Pais: 52 Teorica:5 Brecha:-47,Pais: 43 Teorica:6 Brecha:-37
C450,Pais: 81 Teorica:5 Brecha:-76,Pais: 76 Teorica:5 Brecha:-71,Pais: 75 Teorica:5 Brecha:-70,Pais: 53 Teorica:6 Brecha:-47
C780,Pais: 373 Teorica:1042 Brecha:669,Pais: 379 Teorica:1061 Brecha:682,Pais: 430 Teorica:1081 Brecha:651,Pais: 326 Teorica:1101 Brecha:775


# 1.2 Cálculo de Hospital Basal tomando todos los casos del SSMO

En este apartado, luego de haber identificado los diagnósticos donde existe una brecha de atención,
se estimará la cantidad de casos que deberá atender el INT si es que recibe el 100% de la brecha de pacientes en el SSMO. Dicho en otras palabras, se calculará la necesidad de recursos si es que el INT atiende al 100% de su población en el SSMO.

Para esto, se obtendrá la **población teórica de los diagnósticos donde haya brecha de atención en el país**. Luego, se sumarán estos casos a los egresos INT del 2019. Finalmente, con la cantidad de egresos nuevos, se estimarán los dias de estada necesarios, y la cantidad de camas necesarias para suplir tales días. O sea:

$Población Teórica SSMO (PTSSMO) + Egresos 2019 INT = Casos Nuevos INT$

$Cantidad de Camas Necesarias = \frac{(Casos Nuevos INT * Dias de Estada Promedio Diagnostico)}{365}$

In [13]:
# Los diagnosticos con necesidad sanitaria estan en 
# la variable `diagnoticos_con_necesidad_sanitaria`

# Deja solamente la poblacion teorica del SSMO
poblacion_teorica_ssmo = poblacion_teorica.query("Estrato == 'SSMO'")

# Selecciona los egresos teoricos hasta 2035
poblacion_teorica_ssmo = poblacion_teorica_ssmo[[i for i in range(2017, 2036)]]

# Corrige por la poblacion de FONASA y hospitalizados
poblacion_teorica_ssmo = round(poblacion_teorica_ssmo * RATIO_FONASA * RATIO_HOSPITALIZADOS)

In [14]:
# Aqui se deja un espacio para calcular la brecha a nivel de servicio. Es necesario obtener los
# egresos agrupados a nivel de servicio, para determinar si existe una brecha de atencion a nivel
# de servicio.

# Si se utiliza la brecha nacional, y se obtiene un porcentaje de esta para estimar la brecha del
# servicio, seria una estimacion poco exacta.
HOSPITALES_SSMO = [
    112300,
    112606,
    112609,
    112612,
    112607,
    112608,
    112610,
    112611,
    112613,
    112102,
    112100,
    112101,
    112107,
    114104,
    112104,
    112103,
    112105,
    112106,
]

In [15]:
# Aisla solamente los recintos del SSMO, y calcula los egresos para los diags relevantes
egresos_ssmo = egresos_pais.query("ESTABLECIMIENTO_SALUD.isin(@HOSPITALES_SSMO)")
tabla_dinamica_egresos_ssmo = pd.pivot_table(
    egresos_ssmo,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [16]:
poblacion_teorica_ssmo

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
Diagnostico,,,,,,,,,,,,,,,,,,,
C33X,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
C340,76.0,78.0,81.0,83.0,84.0,85.0,85.0,86.0,86.0,87.0,87.0,88.0,88.0,88.0,89.0,89.0,90.0,90.0,90.0
C341,76.0,78.0,81.0,83.0,84.0,85.0,85.0,86.0,86.0,87.0,87.0,88.0,88.0,88.0,89.0,89.0,90.0,90.0,90.0
C342,76.0,78.0,81.0,83.0,84.0,85.0,85.0,86.0,86.0,87.0,87.0,88.0,88.0,88.0,89.0,89.0,90.0,90.0,90.0
C343,76.0,78.0,81.0,83.0,84.0,85.0,85.0,86.0,86.0,87.0,87.0,88.0,88.0,88.0,89.0,89.0,90.0,90.0,90.0
C381,13.0,14.0,14.0,14.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,16.0,16.0,16.0,16.0
C384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C780,76.0,78.0,81.0,83.0,84.0,85.0,85.0,86.0,86.0,87.0,87.0,88.0,88.0,88.0,89.0,89.0,90.0,90.0,90.0


In [17]:
tabla_dinamica_egresos_ssmo

n_egresos               
ANO_EGRESO      2017 2018 2019 2020
DIAG1                              
C33X               1    1    4    8
C340              22   19   24    7
C341             183  223  221   78
C342              20   24   21    7
C343             120  124  123   39
C381              15   21   19    5
C384               9   10   11    4
C450              23   27    8    5
C780              82   73   78   26
C782              35   63   56   57
D143              32   40   28    7
D381              40   99  105   54
E848              43   47   49   23
I051               7    6    3    6
I080              39   42   43   17
I081              29   44   43   14
I340              46   49   38   28
I350             126  168  156   90
I351              37   52   49   31
I352              36   30   44   26
I420              43   54   54   20
I456              50   29   36   20
I472              43   51   44   37
I495              78   85  117   41
I710              30   39   34   39
I712               9   14   16   31
J398              24   13   21   20
J47X              59   55   47   23
J679              80   72   71   11
J841             134  128  144   75
J848              26   60   61   28
J849              53   28   33   14
J860              22   63   12   13
J869              49   40   39   27
J90X             103  134  113   81
J91X               0    1    0    0
J931              30   47   36   19
J955              78   64   89   57
J980               6    8    2   12
M348              21   29   29    3
Q211              95   62   85   52
Q231               2    2    5   23
Q676              21   24   13   21
T820              11   18   25   28
T821              45   45   47   31
Z450              71  134  139   61